<a href="https://colab.research.google.com/github/noahhansil/BreastCancerClassification/blob/main/BreastCancerClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import statements
import pandas as pd
import numpy as np
!pip install xgboost
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def read_data():
  data = pd.read_csv('data.csv')
  data.replace('M', 1,inplace=True)
  data.replace('B',0, inplace=True)
  data.drop(columns=['Unnamed: 32','id'],inplace=True)
  return data

In [3]:
def normalization(df):
  # normalization calculation is  X - Xmin / Xmax - Xmin'
  normalized_df = df.copy()
  for name, values in normalized_df.iteritems():
    column_name = str(name)
    normalized_df[column_name] = (normalized_df[column_name] - normalized_df[column_name].min()) / (normalized_df[column_name].max() - normalized_df[column_name].min())

  normalized_df['diagnosis'] = normalized_df['diagnosis'].astype(int)
  
  return normalized_df



In [4]:
def train_test_split(df):
  split = int(np.floor(0.8 * len(df)))
  df = df.sample(frac=1,random_state=200)
  x_train = df[:split]
  y_train = df['diagnosis'][:split].values
  x_test = df[split:]
  y_test = df['diagnosis'][split:].values
  print(split)
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  
  return x_train, y_train, x_test, y_test

In [5]:
# using grid search to evaluate: GridSearchCV
def hyperparameter_tuning(x_train, y_train):
  '''
    Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'sub_sample': 0.6}
    Lowest RMSE:  3.211133090353982e-05
  '''

  params = {
      'max_depth': [3, 5, 7, 9],
      'n_estimators': [100, 500, 1000],
      'learning_rate': [.01, 0.05, 0.2, 0.3],
      'colsample_bytree': [0.5, 0.7, 1.0],
      'sub_sample': [0.6, 0.75, 1.0]}

  xgbr = xgb.XGBRegressor(seed = 20)

  clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=1)
  
  clf.fit(x_train, y_train)

  print("Best parameters:", clf.best_params_)
  print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

  return clf.best_params_, -clf.best_score_



In [9]:
def main():

    # read data
    data = read_data()

    # normalization
    normalized_df = normalization(data)

    # Test/Train
    x_train, y_train, x_test, y_test = train_test_split(normalized_df) 

    # hyperParameter tuning
    best_params, rmse = hyperparameter_tuning(x_train, y_train)
    print(best_params, rmse)

In [10]:
main()

455
(455, 31) (455,) (114, 31) (114,)
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[01:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:29:37

In [ ]:
# runtime
'''
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'sub_sample': 0.6}
Lowest RMSE:  4.380097556673973e-05
'''

